In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tinyllama-empathetic-sft-v41/adapter_model.safetensors
/kaggle/input/tinyllama-empathetic-sft-v41/training_args.bin
/kaggle/input/tinyllama-empathetic-sft-v41/adapter_config.json
/kaggle/input/tinyllama-empathetic-sft-v41/README.md
/kaggle/input/tinyllama-empathetic-sft-v41/tokenizer.json
/kaggle/input/tinyllama-empathetic-sft-v41/tokenizer_config.json
/kaggle/input/tinyllama-empathetic-sft-v41/special_tokens_map.json
/kaggle/input/tinyllama-empathetic-sft-v41/tokenizer.model
/kaggle/input/tinyllama-empathetic-sft-v41/checkpoint-186/adapter_model.safetensors
/kaggle/input/tinyllama-empathetic-sft-v41/checkpoint-186/trainer_state.json
/kaggle/input/tinyllama-empathetic-sft-v41/checkpoint-186/training_args.bin
/kaggle/input/tinyllama-empathetic-sft-v41/checkpoint-186/adapter_config.json
/kaggle/input/tinyllama-empathetic-sft-v41/checkpoint-186/README.md
/kaggle/input/tinyllama-empathetic-sft-v41/checkpoint-186/tokenizer.json
/kaggle/input/tinyllama-empathetic-sft-v41/checkp

In [2]:
# Download and extract the dataset
!wget https://dl.fbaipublicfiles.com/parlai/empatheticdialogues/empatheticdialogues.tar.gz
!mkdir -p empatheticdialogues
!tar -xzf empatheticdialogues.tar.gz -C empatheticdialogues

--2025-06-08 08:17:01--  https://dl.fbaipublicfiles.com/parlai/empatheticdialogues/empatheticdialogues.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.44, 18.238.176.19, 18.238.176.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28022709 (27M) [application/gzip]
Saving to: ‘empatheticdialogues.tar.gz’

empatheticdialogues 100%[===================>]  26.72M  55.7MB/s    in 0.5s    

2025-06-08 08:17:01 (55.7 MB/s) - ‘empatheticdialogues.tar.gz’ saved [28022709/28022709]



In [3]:
import pandas as pd
from datasets import Dataset

# Load all CSVs
train_df = pd.read_csv("empatheticdialogues/empatheticdialogues/train.csv", quotechar='"', escapechar='\\', on_bad_lines='skip')
valid_df = pd.read_csv("empatheticdialogues/empatheticdialogues/valid.csv", quotechar='"', escapechar='\\', on_bad_lines='skip')
test_df  = pd.read_csv("empatheticdialogues/empatheticdialogues/test.csv", quotechar='"', escapechar='\\', on_bad_lines='skip')

# Create "Instruction: <context>\nResponse: <utterance>" format
train_df["text"] = train_df.apply(lambda row: f"Instruction: {row['context']}\nResponse: {row['utterance']}", axis=1)
valid_df["text"] = valid_df.apply(lambda row: f"Instruction: {row['context']}\nResponse: {row['utterance']}", axis=1)

# Subset for Colab training
train_sample = train_df.sample(n=5000, random_state=42)
valid_sample = valid_df.sample(n=500, random_state=42)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_sample[["text"]])
val_dataset = Dataset.from_pandas(valid_sample[["text"]])


/tmp/ipykernel_35/990961786.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("empatheticdialogues/empatheticdialogues/train.csv", quotechar='"', escapechar='\\', on_bad_lines='skip')


In [4]:
!pip install -q transformers datasets peft accelerate evaluate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [5]:
import torch
import psutil
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback
from peft import LoraConfig, TaskType, get_peft_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def print_memory():
    if torch.cuda.is_available():
        print(f"[GPU] Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    mem = psutil.virtual_memory()
    print(f"[CPU] Used: {mem.used / 1e9:.2f} GB / {mem.total / 1e9:.2f} GB")


2025-06-08 08:19:03.621649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749370744.082765      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749370744.209850      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


In [6]:
torch.cuda.empty_cache()

In [7]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
#best_conf
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16).to(device)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32, 
    lora_dropout=0.05, 
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, lora_config)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
training_args = TrainingArguments(
    output_dir="./tinyllama-empathetic-sft-v8",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=4e-5, 
    warmup_ratio=0.05,
    weight_decay=0.01,
    gradient_accumulation_steps=2,  
    bf16=True,  
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none"
)

In [10]:
import time

start_time = time.time()  # Start the timer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Starting supervised fine-tuning...")
trainer.train()

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

# Convert to hours, minutes, and seconds for readability
hours, rem = divmod(elapsed_time, 3600)
minutes, seconds = divmod(rem, 60)
print(f"Training completed in {int(hours):02d}:{int(minutes):02d}:{int(seconds):02d} (hh:mm:ss)")


/tmp/ipykernel_35/765275579.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting supervised fine-tuning...


Epoch,Training Loss,Validation Loss
1,0.972900,2.025036
2,0.906200,1.997857


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Training completed in 02:26:25 (hh:mm:ss)


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def print_memory():
    if torch.cuda.is_available():
        print(f"[GPU] Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    mem = psutil.virtual_memory()
    print(f"[CPU] Used: {mem.used / 1e9:.2f} GB / {mem.total / 1e9:.2f} GB")


Using device: cuda


In [19]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "I had a rough day at work and I’m feeling exhausted.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))


Response: <|system|>
You are a helpful assistant.
<|user|>
I had a rough day at work and I’m feeling exhausted.
<|assistant|>
I understand how you feel. It's okay to take a break and rest. You can take a nap or go for a walk to recharge. You can also try to relax and do something that you enjoy. Remember to take care of yourself and give yourself some time to rest.


In [11]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "I feel like no one really understands what I’m going through.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: I'm sorry you're feeling that way. It might help to talk to someone you trust or write your thoughts down. You're not alone, and expressing yourself is a good step.

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Response: <|system|>
You are a helpful assistant.
<|user|>
I feel like no one really understands what I’m going through.
<|assistant|>
I understand that feeling. It can be hard to communicate with people who don't understand what you're going through. It's good to have someone who can help you understand.  I'm glad you're doing well.  I hope you're able to find some peace and solace.  Good luck!  I hope you find the support you need.  I'm sure you'll get through this.  You're a strong person.  You'll get through this. 


In [17]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "My best friend hasn’t talked to me in days, and it’s making me anxious.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: It's understandable to feel anxious. Try reaching out with a simple message — sometimes people get busy, but it doesn't mean they don't care. Open communication helps.

Response: <|system|>
You are a helpful assistant.
<|user|>
My best friend hasn’t talked to me in days, and it’s making me anxious.
<|assistant|>
It sounds like you are feeling anxious about your friend's behavior. It's understandable to feel that way. It's common for people to feel anxious when they don't hear from someone they care about. It's important to remember that it's okay to feel that way and that you're not alone. You can try to reach out to your friend and see if they're okay. If they're not, it's okay to ask for their perspective and try


In [18]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "I’m overwhelmed with school and feel like I’m falling behind.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: Take a deep breath — you're doing your best. Try breaking your tasks into smaller pieces and focus on one thing at a time. Progress happens step by step.

Response: <|system|>
You are a helpful assistant.
<|user|>
I’m overwhelmed with school and feel like I’m falling behind.
<|assistant|>
I understand how you feel. It can be really tough to balance school and other responsibilities. However, it's important to prioritize your time and make sure you're doing the things that are most important to you. It's also helpful to take breaks and prioritize self-care. Remember that you're doing a great job and that you'll get there eventually. Don't give up!


In [16]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "I tried really hard on my project, but it didn’t turn out the way I hoped.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: That can be frustrating. What's important is that you put in the effort — that's how we grow. Try to reflect on what went well and what you could improve next time.

Response: <|system|>
You are a helpful assistant.
<|user|>
I tried really hard on my project, but it didn’t turn out the way I hoped.
<|assistant|>
I'm sorry to hear that. It sounds like you put a lot of effort into it. It's always frustrating when you don't get the results you want. I hope you learned something from the experience and that you can come back to it with a fresh perspective. Good luck!


In [17]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "I got some bad news about a family member today.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: I'm really sorry to hear that. Make sure to take care of yourself and reach out for support if you need it. Talking to a loved one can be very comforting.

Response: <|system|>
You are a helpful assistant.
<|user|>
I got some bad news about a family member today.
<|assistant|>
That's terrible news. I'm sorry to hear that. What happened? Did they pass away? Did they have any children? Did they have any other family members? I hope you're doing okay.


In [20]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "I feel really lonely these days.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: Loneliness can be hard. Try engaging in activities you enjoy or joining a group with shared interests. Connecting with others takes time, but it’s possible.

Response: <|system|>
You are a helpful assistant.
<|user|>
I feel really lonely these days.
<|assistant|>
I understand how you feel. It's hard to be alone sometimes. But you can always reach out to friends or family for support. It's okay to ask for help when you need it.


In [19]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "Everyone expects me to be strong, but I’m really struggling inside.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: It’s okay to feel that way. Even strong people need support. Try opening up to someone you trust — you don’t have to carry everything alone.

Response: <|system|>
You are a helpful assistant.
<|user|>
Everyone expects me to be strong, but I’m really struggling inside.
<|assistant|>
It sounds like you are dealing with some personal issues. It is good to know that you are seeking help. It is always better to seek help when you are struggling. You can talk to someone about it and get the support you need. It is not easy to be strong, but it is important to be strong for yourself and others.


In [20]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "My partner and I had a big argument and now I feel emotionally drained.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: Arguments can be really exhausting. Take some time to cool down and reflect. When you’re ready, try to have a calm, honest conversation about how you feel.

Response: <|system|>
You are a helpful assistant.
<|user|>
My partner and I had a big argument and now I feel emotionally drained.
<|assistant|>
I'm sorry to hear that. It sounds like you need to take a break and spend some time with your partner. It's always better to work things out with your partner rather than going to a therapist. I hope you can work things out and come out of it feeling better.


In [21]:
prompt = (
    "<|system|>\n"
    "You are a helpful assistant.\n"
    "<|user|>\n"
    "I’m scared about what the future holds. Everything feels uncertain.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: It’s natural to feel uncertain about the future. Focus on what you can control today — small steps can help you feel more grounded and confident.

Response: <|system|>
You are a helpful assistant.
<|user|>
I’m scared about what the future holds. Everything feels uncertain.
<|assistant|>
I understand how you feel. It's hard to predict the future but I'm sure you'll be fine. You have a good job and a supportive family. I'm sure you'll be okay. Just be patient and keep working hard. Good luck! 🙂 

I hope you're doing well. 😊


In [18]:
prompt = (
    "<|system|>\n"
    "How would you respond to this empthatically,Respond without using emoticons.\n"
    "<|user|>\n"
    "I just feel like giving up lately.\n"
    "<|assistant|>\n"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=150,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))

# expected: I’m sorry you’re feeling this way. You’re not alone — things can get better with time and support. Consider talking to someone you trust or a professional. You matter.

Response: <|system|>
How would you respond to this empthatically,Respond without using emoticons.
<|user|>
I just feel like giving up lately.
<|assistant|>
I understand that feeling. It's hard to keep going when you're feeling down. But remember that you're not alone. You can always seek help from friends or family. You can also try to find a hobby or activity that you enjoy. It's always better to have something to look forward to than to just give up. Good luck!


In [15]:
trainer.save_model("tinyllama-empathetic-sft-v8")
print("Model saved.")


Model saved.


In [16]:
import shutil

shutil.make_archive("tinyllama-empathetic-sft-v8", 'zip', "tinyllama-empathetic-sft-v8")


'/kaggle/working/tinyllama-empathetic-sft-v8.zip'

In [17]:
import os

os.makedirs("/kaggle/working/output", exist_ok=True)
shutil.move("tinyllama-empathetic-sft-v8.zip", "/kaggle/working/output/")

'/kaggle/working/output/tinyllama-empathetic-sft-v8.zip'

In [12]:
print("Pad token:", tokenizer.pad_token)
print("EOS token:", tokenizer.eos_token)
print("BOS token:", tokenizer.bos_token)
print("UNK token:", tokenizer.unk_token)


Pad token: </s>
EOS token: </s>
BOS token: <s>
UNK token: <unk>


In [42]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import nltk
nltk.download("punkt")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Load models
base_model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16
).to(device)

fine_tuned_model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/input/tinyllama-empathetic-sft-v7", torch_dtype=torch.bfloat16
).to(device)

# BLEU smoother
smoothie = SmoothingFunction().method4

def generate_response(model, prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    output = model.generate(
        input_ids,
        max_length=150,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

def compute_bleu(reference_text, candidate_text):
    reference_tokens = word_tokenize(reference_text)
    candidate_tokens = word_tokenize(candidate_text)
    return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothie)

def evaluate_model_bleu(pairs):
    results = []
    base_total = 0.0
    fine_tuned_total = 0.0

    for i, (prompt, expected_response) in enumerate(pairs):
        base_response = generate_response(base_model, prompt)
        fine_tuned_response = generate_response(fine_tuned_model, prompt)

        base_bleu = compute_bleu(expected_response, base_response)
        fine_tuned_bleu = compute_bleu(expected_response, fine_tuned_response)

        base_total += base_bleu
        fine_tuned_total += fine_tuned_bleu

        results.append({
            "index": i,
            "prompt": prompt,
            "expected_response": expected_response,
            "base_response": base_response,
            "fine_tuned_response": fine_tuned_response,
            "base_bleu": base_bleu,
            "fine_tuned_bleu": fine_tuned_bleu
        })

    avg_base_bleu = base_total / len(pairs)
    avg_fine_tuned_bleu = fine_tuned_total / len(pairs)

    return results, avg_base_bleu, avg_fine_tuned_bleu


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
pairs = [
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI feel like no one really understands what I’m going through.\n"
        "<|assistant|>\n",
        "I'm really sorry you're feeling that way. It can be incredibly hard when it seems like no one truly gets what you're going through. You're not alone, and your feelings are valid. I'm here to listen and support you."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nMy best friend hasn’t talked to me in days, and it’s making me anxious.\n"
        "<|assistant|>\n",
        "That sounds really tough. It’s painful when someone close goes quiet, and it’s natural to feel anxious. Remember, their silence doesn’t define your worth. You deserve care and connection, and I’m here for you whenever you want to talk."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI’m overwhelmed with school and feel like I’m falling behind.\n"
        "<|assistant|>\n",
        "I hear you, it’s so overwhelming when school piles up and you feel like you’re slipping behind. It’s okay to feel this way, and it doesn’t mean you’re failing. Take a deep breath, break things into smaller steps, and remember you’re doing your best. You’ve got this."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI tried really hard on my project, but it didn’t turn out the way I hoped.\n"
        "<|assistant|>\n",
        "That’s really disappointing, and I’m sorry your project didn’t go as you hoped. Your effort matters so much, and setbacks don’t erase all the hard work you put in. Every experience is a step forward, even if it doesn’t feel that way now. Keep believing in yourself."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI got some bad news about a family member today.\n"
        "<|assistant|>\n",
        "I’m so sorry to hear that. Receiving tough news about someone you love can feel overwhelming and heavy. It’s okay to feel whatever comes up right now, grief, fear, confusion. You’re not alone, and I’m here to support you through this."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI feel really lonely these days.\n"
        "<|assistant|>\n",
        "Feeling lonely can be so painful, and I’m sorry you’re going through this. Remember, your feelings matter, and it’s okay to reach out, even just a little. You’re not alone, and there are people who care deeply about you. I’m here to listen whenever you need."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nEveryone expects me to be strong, but I’m really struggling inside.\n"
        "<|assistant|>\n",
        "That’s incredibly hard, to carry the weight of others’ expectations while struggling inside. It’s okay to admit you’re not okay. Your strength includes being honest about your feelings, and you don’t have to face this alone. You deserve support and kindness."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nMy partner and I had a big argument and now I feel emotionally drained.\n"
        "<|assistant|>\n",
        "I’m really sorry you’re feeling this way. Arguments with someone you care about can leave you feeling exhausted and hurt. It’s okay to take time to breathe and care for yourself. Your feelings are valid, and healing takes time. You’re not alone."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI’m scared about what the future holds. Everything feels uncertain.\n"
        "<|assistant|>\n",
        "Feeling scared about the future is completely natural when everything feels uncertain. It’s okay to not have all the answers right now. Take things one step at a time, and be gentle with yourself. You’re stronger than you realize, and I’m here with you through this."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI just feel like giving up lately.\n"
        "<|assistant|>\n",
        "I’m really sorry you’re feeling this way. It’s okay to feel overwhelmed and unsure, but please remember you’re not alone and your feelings matter. Reaching out for support can make a difference, you deserve kindness and hope, even when it’s hard to see."
    ),
]

results, avg_base, avg_fine_tuned = evaluate_model_bleu(pairs)

print("Base Avg BLEU:", avg_base)
print("Fine-tuned Avg BLEU:", avg_fine_tuned)


Base Avg BLEU: 0.031318432631349154
Fine-tuned Avg BLEU: 0.038012422266763636


In [21]:
!pip install bert-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.5 MB/s eta 0:00:00a 0:00:01


In [22]:
from bert_score import score as bert_score

def compute_bertscore(reference_texts, candidate_texts, lang="en"):
    # reference_texts and candidate_texts should be lists of strings
    P, R, F1 = bert_score(candidate_texts, reference_texts, lang=lang, verbose=False)
    return P.tolist(), R.tolist(), F1.tolist()

def evaluate_model_bertscore(pairs):
    base_preds = []
    fine_tuned_preds = []
    refs = []

    for prompt, expected_response in pairs:
        base_response = generate_response(base_model, prompt)
        fine_tuned_response = generate_response(fine_tuned_model, prompt)

        refs.append(expected_response)
        base_preds.append(base_response)
        fine_tuned_preds.append(fine_tuned_response)

    base_P, base_R, base_F1 = compute_bertscore(refs, base_preds)
    fine_P, fine_R, fine_F1 = compute_bertscore(refs, fine_tuned_preds)

    results = []
    for i in range(len(pairs)):
        results.append({
            "index": i,
            "prompt": pairs[i][0],
            "expected_response": refs[i],
            "base_response": base_preds[i],
            "fine_tuned_response": fine_tuned_preds[i],
            "base_bertscore": base_F1[i],
            "fine_tuned_bertscore": fine_F1[i]
        })

    avg_base_bert = sum(base_F1) / len(base_F1)
    avg_fine_bert = sum(fine_F1) / len(fine_F1)

    return results, avg_base_bert, avg_fine_bert


In [44]:
bert_results, avg_base_bert, avg_fine_bert = evaluate_model_bertscore(pairs)

print(f"Average Base BERTScore: {avg_base_bert:.4f}")
print(f"Average Fine-Tuned BERTScore: {avg_fine_bert:.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average Base BERTScore: 0.8467
Average Fine-Tuned BERTScore: 0.8500


In [24]:
def collect_model_responses_for_human_eval(pairs, base_model, fine_tuned_model, tokenizer, device):

    results = []

    for i, (prompt, expected_response) in enumerate(pairs):
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

        # Generate from base model
        base_output = base_model.generate(
            input_ids,
            max_length=150,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
        base_response = tokenizer.decode(base_output[0], skip_special_tokens=True)

        # Generate from fine-tuned model
        fine_output = fine_tuned_model.generate(
            input_ids,
            max_length=150,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
        fine_response = tokenizer.decode(fine_output[0], skip_special_tokens=True)

        # Append to results
        results.append({
            "index": i,
            "prompt": prompt,
            "expected_response": expected_response,
            "base_response": base_response,
            "fine_tuned_response": fine_response
        })

    return results


In [46]:
human_eval_data = collect_model_responses_for_human_eval(
    pairs=pairs,
    base_model=base_model,
    fine_tuned_model=fine_tuned_model,
    tokenizer=tokenizer,
    device=device
)

In [47]:
# Example: Print nicely for human review
for item in human_eval_data:
    print(f"\n--- Example {item['index'] + 1} ---")
    print(f"Prompt:\n{item['prompt']}")
    print(f"\nExpected Response:\n{item['expected_response']}")
    print(f"\nBase Model Response:\n{item['base_response']}")
    print(f"\nFine-tuned Model Response:\n{item['fine_tuned_response']}")
    print("\n" + "-" * 60)



--- Example 1 ---
Prompt:
<|system|>
How would you respond to this empthatically,Respond without using emoticons.
<|user|>
I feel like no one really understands what I’m going through.
<|assistant|>


Expected Response:
I'm really sorry you're feeling that way. It can be incredibly hard when it seems like no one truly gets what you're going through. You're not alone, and your feelings are valid. I'm here to listen and support you.

Base Model Response:
<|system|>
How would you respond to this empthatically,Respond without using emoticons.
<|user|>
I feel like no one really understands what I’m going through.
<|assistant|>
I understand your frustration. It can be challenging to express emotions in a way that others can understand. However, it’s essential to remember that everyone experiences pain and suffering in their own unique way. It’s okay to feel overwhelmed and alone, but it’s also important to seek support and guidance from trusted friends, family, or mental health professional